In [ ]:
import os
import boto3
from botocore.config import Config

# -------- CONFIGURATION --------
WASABI_ACCESS_KEY = "VIH97PD2FNFFMYQAESDR"
WASABI_SECRET_KEY = "X2UgOtGwJP5DFYnYjSloe9ilVVb89wseGiRKZa9T"
WASABI_REGION = "eu-east-1"  # or your region
BUCKET_NAME = "geoguesser-dataset"
DEST_DIR = "./dataset"  # Local destination folder
# --------------------------------

# Create session with Wasabi (S3-compatible)
session = boto3.session.Session()

s3 = session.client(
    service_name='s3',
    region_name=WASABI_REGION,
    aws_access_key_id=WASABI_ACCESS_KEY,
    aws_secret_access_key=WASABI_SECRET_KEY,
    endpoint_url=f'https://s3.{WASABI_REGION}.wasabisys.com',  # Wasabi endpoint
    config=Config(signature_version='s3v4')
)

def download_all_files():
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=BUCKET_NAME)

    os.makedirs(DEST_DIR, exist_ok=True)

    for page in pages:
        for obj in page.get('Contents', []):
            key = obj['Key']
            local_path = os.path.join(DEST_DIR, key)

            os.makedirs(os.path.dirname(local_path), exist_ok=True)
            print(f"Downloading: {key} -> {local_path}")
            s3.download_file(BUCKET_NAME, key, local_path)
